## Load data

In [5]:
import pandas as pd 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

In [6]:
IS_LOCAL = True

import os

if IS_LOCAL:
    PATH = r"/Users/trinhthilananh/Documents/Feature Engineering/home-credit-default-risk"
else:
    PATH = "../input"

print(os.listdir(PATH))


['test_final_selected_features.csv', 'application_test.csv', '.DS_Store', 'HomeCredit_columns_description.csv', 'POS_CASH_balance.csv', 'credit_card_balance.csv', 'installments_payments.csv', 'application_train.csv', 'bureau.csv', 'train_final_selected_features.csv', 'previous_application.csv', 'bureau_balance.csv', 'sample_submission.csv']


In [7]:
application_train = pd.read_csv(PATH+"/application_train.csv")
application_test = pd.read_csv(PATH+"/application_test.csv")
bureau = pd.read_csv(PATH+"/bureau.csv")
bureau_balance = pd.read_csv(PATH+"/bureau_balance.csv")
credit_card_balance = pd.read_csv(PATH+"/credit_card_balance.csv")
installments_payments = pd.read_csv(PATH+"/installments_payments.csv")
previous_application = pd.read_csv(PATH+"/previous_application.csv")
POS_CASH_balance = pd.read_csv(PATH+"/POS_CASH_balance.csv")

## Loading Libraries

In [8]:
#importing Useful DataStructures
import pandas as pd
import numpy as np
from scipy.stats import uniform

#importing plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
from prettytable import PrettyTable

#importing Misc Libraries
import os
import gc
import pickle
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime

#for 100% jupyter notebook cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.calibration import CalibratedClassifierCV


## 1. Xử lý bảng application_train.csv và application_test.csv

In [9]:
# Lưu lại ID của tập test để dùng ở cuối
test_ids = application_test['SK_ID_CURR']

# Thêm cột TARGET vào tập test (với giá trị NaN) để gộp 2 file
application_test['TARGET'] = np.nan

# Gộp train và test
df_app = pd.concat([application_train, application_test], ignore_index=True)

print(f"Kích thước df_app (gộp): {df_app.shape}")

# --- Feature Engineering Cơ bản ---

# 1. Làm sạch dữ liệu bất thường (Anomaly)
df_app['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
df_app['CODE_GENDER'].replace('XNA', np.nan, inplace=True)

# 2. Tạo các thuộc tính tỷ lệ (Domain Features)
# Thêm 0.00001 để tránh lỗi chia cho 0
df_app['CREDIT_INCOME_RATIO'] = df_app['AMT_CREDIT'] / (df_app['AMT_INCOME_TOTAL'] + 0.00001)
df_app['ANNUITY_INCOME_RATIO'] = df_app['AMT_ANNUITY'] / (df_app['AMT_INCOME_TOTAL'] + 0.00001)
df_app['CREDIT_ANNUITY_RATIO'] = df_app['AMT_CREDIT'] / (df_app['AMT_ANNUITY'] + 0.00001)
df_app['CREDIT_GOODS_PRICE_RATIO'] = df_app['AMT_CREDIT'] / (df_app['AMT_GOODS_PRICE'] + 0.00001)
df_app['DAYS_EMPLOYED_TO_BIRTH_RATIO'] = df_app['DAYS_EMPLOYED'] / (df_app['DAYS_BIRTH'] + 0.00001)

# 3. Kết hợp các thuộc tính EXT_SOURCE
df_app['EXT_SOURCES_MEAN'] = df_app[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
df_app['EXT_SOURCES_PROD'] = df_app['EXT_SOURCE_1'] * df_app['EXT_SOURCE_2'] * df_app['EXT_SOURCE_3']
df_app['EXT_SOURCES_MIN'] = df_app[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)
df_app['EXT_SOURCES_MAX'] = df_app[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)

# 4. Xử lý các cột Categorical (dùng One-Hot Encoding)
# Lấy danh sách các cột 'object'
categorical_cols = df_app.select_dtypes(include=['object']).columns

# Dùng pd.get_dummies
df_app = pd.get_dummies(df_app, columns=categorical_cols, dummy_na=True)

print(f"Kích thước df_app sau khi xử lý: {df_app.shape}")

Kích thước df_app (gộp): (356255, 122)
Kích thước df_app sau khi xử lý: (356255, 270)


## 2. Xử lý bureau.csv và bureau_balance.csv

In [10]:
print("2.1: Xử lý bureau_balance")

# 1. Xử lý cột STATUS (Chạy get_dummies một cách rõ ràng)
# Chúng ta sẽ lưu lại tên các cột mới được tạo
original_cols = set(bureau_balance.columns)

if 'STATUS' in bureau_balance.columns:
    print("Đang chạy pd.get_dummies cho cột 'STATUS'...")
    # Thêm dtype=int để đảm bảo các cột mới là SỐ
    bureau_balance = pd.get_dummies(bureau_balance, columns=['STATUS'], dummy_na=True, dtype=int)
    # Lấy danh sách các cột mới
    new_status_cols = list(set(bureau_balance.columns) - original_cols)
else:
    print("Cột 'STATUS' dường như đã được get_dummies. Đang tìm các cột STATUS_...")
    new_status_cols = [col for col in bureau_balance.columns if col.startswith('STATUS_')]

# Kiểm tra xem có cột nào để tổng hợp không
if not new_status_cols:
    print("!!! LỖI: Không tìm thấy cột STATUS nào (STATUS_0, STATUS_C...) để tổng hợp.")
    raise ValueError("Không tìm thấy cột STATUS_... để tổng hợp. Vui lòng kiểm tra lại file bureau_balance.")

print(f"Các cột STATUS sẽ được tổng hợp: {new_status_cols}")

# 2. Định nghĩa các phép tổng hợp
# Chỉ tổng hợp trên các cột STATUS mới
balance_aggregations = {}
for col in new_status_cols:
    balance_aggregations[col] = ['mean', 'min', 'max', 'sum', 'count']

# 3. Tổng hợp (Aggregate) theo SK_ID_BUREAU
# Dòng này sẽ hoạt động vì balance_aggregations KHÔNG rỗng
agg_bureau_balance = bureau_balance.groupby('SK_ID_BUREAU').agg(balance_aggregations)

# 4. GIẢI PHÁP SỬA LỖI (MultiIndex) - Giữ nguyên như trước
# Chuyển 2 cấp độ cột (ví dụ: ('STATUS_0', 'MEAN')) về 1 cấp độ ('STATUS_0_MEAN')
agg_bureau_balance.columns = ['_'.join(col).upper() for col in agg_bureau_balance.columns.values]
agg_bureau_balance.reset_index(inplace=True) # Đưa SK_ID_BUREAU về làm cột

print(f"Kích thước agg_bureau_balance (đã agg): {agg_bureau_balance.shape}")
print(f"5 cột đầu tiên: {list(agg_bureau_balance.columns[:5])}")

2.1: Xử lý bureau_balance
Đang chạy pd.get_dummies cho cột 'STATUS'...
Các cột STATUS sẽ được tổng hợp: ['STATUS_nan', 'STATUS_C', 'STATUS_2', 'STATUS_0', 'STATUS_1', 'STATUS_X', 'STATUS_3', 'STATUS_5', 'STATUS_4']
Kích thước agg_bureau_balance (đã agg): (817395, 46)
5 cột đầu tiên: ['SK_ID_BUREAU', 'STATUS_NAN_MEAN', 'STATUS_NAN_MIN', 'STATUS_NAN_MAX', 'STATUS_NAN_SUM']


In [11]:
print("2.2: Xử lý bureau...")

# 1. Hợp nhất (merge) bureau với agg_bureau_balance đã tạo ở bước 2.1
df_bureau = bureau.merge(agg_bureau_balance, on='SK_ID_BUREAU', how='left')

# 2. Xử lý categorical trong bureau
bureau_categorical_cols = df_bureau.select_dtypes(include=['object']).columns
df_bureau = pd.get_dummies(df_bureau, columns=bureau_categorical_cols, dummy_na=True)

# 3. Định nghĩa các phép tổng hợp
# Lấy các cột số (numeric)
numeric_aggregations = {
    'DAYS_CREDIT': ['min', 'max', 'mean', 'count', 'std'], 
    'CREDIT_DAY_OVERDUE': ['mean', 'max', 'std'], 
    'AMT_CREDIT_SUM': ['sum', 'mean', 'max', 'std'], 
    'AMT_CREDIT_SUM_DEBT': ['sum', 'mean', 'std'], 
    'AMT_CREDIT_SUM_OVERDUE': ['sum', 'mean', 'std'],
    'CNT_CREDIT_PROLONG': ['sum', 'mean', 'std'], 
}
# Lấy các cột categorical đã one-hot (từ bureau và bureau_balance)
categorical_cols = [col for col in df_bureau.columns if col.startswith(('CREDIT_ACTIVE_', 'CREDIT_CURRENCY_', 'CREDIT_TYPE_', 'STATUS_'))]
for col in categorical_cols:
    numeric_aggregations[col] = ['mean', 'sum'] # Lấy tỷ lệ và số lượng

# 4. Tổng hợp (Aggregate) theo SK_ID_CURR
agg_bureau = df_bureau.groupby('SK_ID_CURR').agg(numeric_aggregations)

# 5. ÁP DỤNG GIẢI PHÁP SỬA LỖI (Đổi tên cột MultiIndex)
agg_bureau.columns = ['BUREAU_' + '_'.join(col).upper() for col in agg_bureau.columns.values]
agg_bureau.reset_index(inplace=True) # SK_ID_CURR về làm cột

print(f"Kích thước agg_bureau (đã agg): {agg_bureau.shape}")
print(f"5 cột đầu tiên: {list(agg_bureau.columns[:5])}")

2.2: Xử lý bureau...
Kích thước agg_bureau (đã agg): (305811, 164)
5 cột đầu tiên: ['SK_ID_CURR', 'BUREAU_DAYS_CREDIT_MIN', 'BUREAU_DAYS_CREDIT_MAX', 'BUREAU_DAYS_CREDIT_MEAN', 'BUREAU_DAYS_CREDIT_COUNT']


## 3. Xử lý Previous_applicaiton.csv

In [12]:
print("3: Xử lý previous_application...")

# 1. Làm sạch giá trị 365243
previous_application['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace=True)
previous_application['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace=True)
previous_application['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace=True)
previous_application['DAYS_LAST_DUE'].replace(365243, np.nan, inplace=True)
previous_application['DAYS_TERMINATION'].replace(365243, np.nan, inplace=True)

# 2. Tạo thuộc tính mới
previous_application['AMT_CREDIT_GOODS_PRICE_DIFF'] = previous_application['AMT_CREDIT'] - previous_application['AMT_GOODS_PRICE']

# 3. Xử lý categorical
prev_categorical_cols = previous_application.select_dtypes(include=['object']).columns
previous_application = pd.get_dummies(previous_application, columns=prev_categorical_cols, dummy_na=True)

# 4. Định nghĩa các phép tổng hợp
prev_aggregations = {
    'AMT_ANNUITY': ['min', 'max', 'mean', 'sum', 'std'],
    'AMT_APPLICATION': ['min', 'max', 'mean'],
    'AMT_CREDIT': ['min', 'max', 'mean', 'sum', 'std'],
    'AMT_CREDIT_GOODS_PRICE_DIFF': ['mean', 'max', 'sum', 'std'],
    'AMT_DOWN_PAYMENT': ['mean', 'sum', 'std'],
    'DAYS_DECISION': ['min', 'max', 'mean', 'std'],
    'CNT_PAYMENT': ['mean', 'sum', 'std'],
}

# Lấy các cột categorical (ví dụ: Tỷ lệ các hợp đồng bị từ chối/chấp nhận)
cat_cols = [col for col in previous_application.columns if col.startswith(('NAME_CONTRACT_STATUS_', 'CODE_REJECT_REASON_'))]
for col in cat_cols:
    prev_aggregations[col] = ['mean', 'sum']
    
# 5. Tổng hợp (Aggregate) theo SK_ID_CURR
agg_previous = previous_application.groupby('SK_ID_CURR').agg(prev_aggregations)

# 6. ÁP DỤNG GIẢI PHÁP SỬA LỖI (Đổi tên cột MultiIndex)
agg_previous.columns = ['PREV_' + '_'.join(col).upper() for col in agg_previous.columns.values]
agg_previous.reset_index(inplace=True) # SK_ID_CURR về làm cột

print(f"Kích thước agg_previous (đã agg): {agg_previous.shape}")

3: Xử lý previous_application...
Kích thước agg_previous (đã agg): (338857, 58)


## 4. Xử lý installments_payments.csv

In [13]:
print("4. Xử lý installments_payments...")

# 1. Tạo thuộc tính mới (Số ngày trả chậm, Số tiền trả thiếu)
installments_payments['DAYS_PAYMENT_DIFF'] = installments_payments['DAYS_INSTALMENT'] - installments_payments['DAYS_ENTRY_PAYMENT']
installments_payments['AMT_PAYMENT_DIFF'] = installments_payments['AMT_INSTALMENT'] - installments_payments['AMT_PAYMENT']

# 2. Định nghĩa tổng hợp
install_aggregations = {
    'NUM_INSTALMENT_VERSION': ['nunique'],
    'DAYS_PAYMENT_DIFF': ['mean', 'max', 'sum', 'std'],
    'AMT_PAYMENT_DIFF': ['mean', 'max', 'sum', 'std'],
    'AMT_INSTALMENT': ['mean', 'sum', 'std'],
    'AMT_PAYMENT': ['mean', 'sum', 'std'],
    'SK_ID_PREV': ['nunique'] # Đếm số khoản vay
}

# 3. Tổng hợp
agg_installments = installments_payments.groupby('SK_ID_CURR').agg(install_aggregations)

# 4. ÁP DỤNG GIẢI PHÁP SỬA LỖI
agg_installments.columns = ['INSTALL_' + '_'.join(col).upper() for col in agg_installments.columns.values]
agg_installments.reset_index(inplace=True)

print(f"Kích thước agg_installments (đã agg): {agg_installments.shape}")

4. Xử lý installments_payments...
Kích thước agg_installments (đã agg): (339587, 17)


## 5. Xử lý credit_card_balance.csv

In [14]:
print("5: Xử lý credit_card_balance...")

# 1. Tạo thuộc tính mới
credit_card_balance['BALANCE_LIMIT_RATIO'] = credit_card_balance['AMT_BALANCE'] / (credit_card_balance['AMT_CREDIT_LIMIT_ACTUAL'] + 0.00001)

# 2. Định nghĩa tổng hợp
cc_aggregations = {
    'AMT_BALANCE': ['mean', 'max', 'sum', 'std'],
    'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'max', 'std'],
    'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'max', 'sum', 'std'],
    'AMT_DRAWINGS_CURRENT': ['mean', 'max', 'sum', 'std'],
    'CNT_DRAWINGS_ATM_CURRENT': ['mean', 'max', 'sum', 'std'],
    'CNT_INSTALMENT_MATURE_CUM': ['max', 'mean', 'std'],
    'SK_DPD': ['mean', 'max', 'sum', 'std'], # Số ngày quá hạn
    'SK_DPD_DEF': ['mean', 'max', 'sum', 'std'], # Số ngày quá hạn (nghiêm trọng)
    'BALANCE_LIMIT_RATIO': ['mean', 'max', 'std']
}

# 3. Tổng hợp
agg_credit_card = credit_card_balance.groupby('SK_ID_CURR').agg(cc_aggregations)

# 4. ÁP DỤNG GIẢI PHÁP SỬA LỖI
agg_credit_card.columns = ['CC_' + '_'.join(col).upper() for col in agg_credit_card.columns.values]
agg_credit_card.reset_index(inplace=True)

print(f"Kích thước agg_credit_card (đã agg): {agg_credit_card.shape}")

5: Xử lý credit_card_balance...
Kích thước agg_credit_card (đã agg): (103558, 34)


## 6. Xử lý POSH_CASH_balance.csv

In [15]:
print("6: Xử lý POS_CASH_balance (Phiên bản sửa lỗi KeyError)")

# 1. Xử lý categorical (Một cách an toàn)
# KIỂM TRA trước khi chạy get_dummies
if 'NAME_CONTRACT_STATUS' in POS_CASH_balance.columns:
    print("Đang chạy pd.get_dummies cho cột 'NAME_CONTRACT_STATUS'...")
    POS_CASH_balance = pd.get_dummies(POS_CASH_balance, columns=['NAME_CONTRACT_STATUS'], dummy_na=True)
else:
    print("Cột 'NAME_CONTRACT_STATUS' dường như đã được get_dummies. Bỏ qua.")
# 2. Định nghĩa tổng hợp
pos_aggregations = {
    'MONTHS_BALANCE': ['min', 'max', 'mean', 'std'], 
    'CNT_INSTALMENT': ['min', 'max', 'mean', 'std'], 
    'CNT_INSTALMENT_FUTURE': ['min', 'max', 'mean', 'std'], 
    'SK_DPD': ['mean', 'max', 'sum', 'std'], 
    'SK_DPD_DEF': ['mean', 'max', 'sum', 'std'], 
}

# Lấy các cột categorical (đã được one-hot)
# Dòng này sẽ tìm các cột 'NAME_CONTRACT_STATUS_Active' v.v.
cat_cols = [col for col in POS_CASH_balance.columns if col.startswith('NAME_CONTRACT_STATUS_')]
for col in cat_cols:
    pos_aggregations[col] = ['mean', 'sum']

# 3. Tổng hợp
agg_pos_cash = POS_CASH_balance.groupby('SK_ID_CURR').agg(pos_aggregations)

# 4. ÁP DỤNG GIẢI PHÁP SỬA LỖI (MultiIndex)
agg_pos_cash.columns = ['POS_' + '_'.join(col).upper() for col in agg_pos_cash.columns.values]
agg_pos_cash.reset_index(inplace=True)

print(f"Kích thước agg_pos_cash (đã agg): {agg_pos_cash.shape}")

6: Xử lý POS_CASH_balance (Phiên bản sửa lỗi KeyError)
Đang chạy pd.get_dummies cho cột 'NAME_CONTRACT_STATUS'...
Kích thước agg_pos_cash (đã agg): (337252, 41)


## 7. Merge các bảng dữ liệu

In [16]:
print("7: Hợp nhất (Merge) tất cả...")

# 1. Hợp nhất df_app (từ Bước 1) với agg_bureau (từ Bước 2.2)
df_final = df_app.merge(agg_bureau, on='SK_ID_CURR', how='left')
print(f"Kích thước sau khi merge bureau: {df_final.shape}")

# 2. Hợp nhất với agg_previous (từ Bước 2.3)
df_final = df_final.merge(agg_previous, on='SK_ID_CURR', how='left')
print(f"Kích thước sau khi merge previous: {df_final.shape}")

# 3. Hợp nhất với agg_installments (từ Bước 2.4)
df_final = df_final.merge(agg_installments, on='SK_ID_CURR', how='left')
print(f"Kích thước sau khi merge installments: {df_final.shape}")

# 4. Hợp nhất với agg_credit_card (từ Bước 2.5)
df_final = df_final.merge(agg_credit_card, on='SK_ID_CURR', how='left')
print(f"Kích thước sau khi merge credit card: {df_final.shape}")

# 5. Hợp nhất với agg_pos_cash (từ Bước 2.6)
df_final = df_final.merge(agg_pos_cash, on='SK_ID_CURR', how='left')
print(f"Kích thước sau khi merge POS cash: {df_final.shape}")

print("--- HỢP NHẤT THÀNH CÔNG, df_final ĐÃ ĐƯỢC TẠO ---")

7: Hợp nhất (Merge) tất cả...
Kích thước sau khi merge bureau: (356255, 433)
Kích thước sau khi merge previous: (356255, 490)
Kích thước sau khi merge installments: (356255, 506)
Kích thước sau khi merge credit card: (356255, 539)
Kích thước sau khi merge POS cash: (356255, 579)
--- HỢP NHẤT THÀNH CÔNG, df_final ĐÃ ĐƯỢC TẠO ---


## 8. Tạo Interaction Features

In [17]:
print("Bắt đầu Bước 4: Tạo các thuộc tính tương tác (Interaction Features)...")

# Tương tác Thu nhập và Khoản vay hiện tại
df_final['APP_INCOME_CREDIT_RATIO'] = df_final['AMT_INCOME_TOTAL'] / (df_final['AMT_CREDIT'] + 0.00001)
df_final['APP_INCOME_ANNUITY_RATIO'] = df_final['AMT_INCOME_TOTAL'] / (df_final['AMT_ANNUITY'] + 0.00001)
df_final['APP_PAYMENT_RATE'] = df_final['AMT_ANNUITY'] / (df_final['AMT_CREDIT'] + 0.00001) # Tỷ lệ trả góp

# Tương tác Thu nhập và Lịch sử Bureau (khoản vay cũ ở nơi khác)
df_final['BUREAU_INCOME_CREDIT_RATIO'] = df_final['BUREAU_AMT_CREDIT_SUM_SUM'] / (df_final['AMT_INCOME_TOTAL'] + 0.00001)
df_final['BUREAU_INCOME_DEBT_RATIO'] = df_final['BUREAU_AMT_CREDIT_SUM_DEBT_SUM'] / (df_final['AMT_INCOME_TOTAL'] + 0.00001)

# Tương tác Thu nhập và Lịch sử Previous App (khoản vay cũ ở Home Credit)
df_final['PREV_INCOME_CREDIT_RATIO'] = df_final['PREV_AMT_CREDIT_SUM'] / (df_final['AMT_INCOME_TOTAL'] + 0.00001)

# Tương tác Thu nhập và Lịch sử Trả góp
df_final['INSTALL_INCOME_PAYMENT_RATIO'] = df_final['INSTALL_AMT_PAYMENT_SUM'] / (df_final['AMT_INCOME_TOTAL'] + 0.00001)

print(f"Kích thước df_final sau khi thêm interaction features: {df_final.shape}")

Bắt đầu Bước 4: Tạo các thuộc tính tương tác (Interaction Features)...
Kích thước df_final sau khi thêm interaction features: (356255, 586)


## 9. Feature Selection

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import re

In [19]:
print("Lọc Feature")

# Tách train_df tạm thời ĐỂ FIT bộ lọc (tránh data leakage)
# df_final lúc này đã có 586 features (bao gồm cả interaction)
train_df_for_selection = df_final[df_final['TARGET'].notnull()].copy()

# === 1. Lọc các cột thiếu quá nhiều dữ liệu ===
missing_threshold = 0.8 # Ngưỡng 80%
missing_counts = train_df_for_selection.isnull().mean()
cols_to_drop_missing = missing_counts[missing_counts > missing_threshold].index

# Áp dụng lọc trên df_final
df_final = df_final.drop(columns=cols_to_drop_missing)
print(f"Đã loại bỏ {len(cols_to_drop_missing)} cột do thiếu dữ liệu > 80%.")


# === 2. Lọc các cột có giá trị duy nhất (ít biến thiên) ===

# Cập nhật lại train_df_for_selection sau khi đã drop ở bước 1
train_df_for_selection = df_final[df_final['TARGET'].notnull()].copy()

unique_counts = train_df_for_selection.nunique()
cols_to_drop_unique = unique_counts[unique_counts == 1].index

# Áp dụng lọc trên df_final
df_final = df_final.drop(columns=cols_to_drop_unique)
print(f"Đã loại bỏ {len(cols_to_drop_unique)} cột do chỉ có 1 giá trị duy nhất.")


print(f"--- Kích thước df_final sau Lọc 'Thô': {df_final.shape} ---")

Lọc Feature
Đã loại bỏ 6 cột do thiếu dữ liệu > 80%.
Đã loại bỏ 35 cột do chỉ có 1 giá trị duy nhất.
--- Kích thước df_final sau Lọc 'Thô': (356255, 545) ---


In [20]:
print("Đã import RandomForestClassifier và SelectFromModel từ sklearn.")

# 1. Tách lại tập train (chỉ train mới có TARGET)
train_df = df_final[df_final['TARGET'].notnull()].copy()
print(f"Kích thước tập train để chọn lọc: {train_df.shape}")

# 2. Xử lý tên cột (cho an toàn)
train_df.columns = [re.sub(r'[^A-Za-z0-9_]+', '', col) for col in train_df.columns]

# 3. Định nghĩa X (features) và y (target)
y_train = train_df['TARGET']
cols_to_drop = ['TARGET', 'SK_ID_CURR']
X_train = train_df.drop(columns=cols_to_drop)

print(f"Số lượng features ban đầu: {X_train.shape[1]}")

# 4. XỬ LÝ NaN/Infinity (BẮT BUỘC cho RandomForest)
X_train.replace([np.inf, -np.inf], 0, inplace=True)
X_train.fillna(0, inplace=True)

print("Đã xử lý NaN và Infinity cho X_train.")

Đã import RandomForestClassifier và SelectFromModel từ sklearn.
Kích thước tập train để chọn lọc: (307511, 545)
Số lượng features ban đầu: 543
Đã xử lý NaN và Infinity cho X_train.


In [21]:
print("9.1: Train RandomForest Selector...")

# 1. Khởi tạo RandomForest
rf_selector = RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    random_state=42
)

print("Đang training mô hình RandomForest... (Có thể mất vài phút)")
rf_selector.fit(X_train, y_train)
print("Training hoàn tất.")

9.1: Train RandomForest Selector...
Đang training mô hình RandomForest... (Có thể mất vài phút)
Training hoàn tất.


In [22]:
print("9.2: Chọn lọc tự động (Threshold='median')...")

# 1. Khởi tạo SelectFromModel
# Đưa mô hình đã train (rf_selector) vào
# 'threshold="median"': Tự động tính toán độ quan trọng trung vị 
#                       và chỉ giữ lại các feature có độ quan trọng cao hơn.
# 'prefit=True': Báo cho SelectFromModel biết chúng ta đã train rf_selector rồi.
selector = SelectFromModel(rf_selector, threshold="median", prefit=True)

print(f"Đã tạo bộ chọn (selector) với ngưỡng = 'median'.")

# 2. Lấy danh sách các feature quan trọng
# .get_support() trả về một mảng True/False cho mỗi feature
feature_mask = selector.get_support()
# Lấy tên của các feature được chọn (là True)
important_features = X_train.columns[feature_mask]

# Chuyển về dạng list
important_features = important_features.tolist()

print(f"\n--- Phân tích chọn lọc ---")
print(f"Tổng số features ban đầu: {X_train.shape[1]}")
print(f"Số lượng features quan trọng (trên mức trung vị) được giữ lại: {len(important_features)}")

print("\n--- Features được chọn: ---")
# Chúng ta có thể xem lại DataFrame cũ để thấy
features_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_selector.feature_importances_
}).sort_values(by='importance', ascending=False)
print(features_df[features_df['feature'].isin(important_features)])

9.2: Chọn lọc tự động (Threshold='median')...
Đã tạo bộ chọn (selector) với ngưỡng = 'median'.

--- Phân tích chọn lọc ---
Tổng số features ban đầu: 543
Số lượng features quan trọng (trên mức trung vị) được giữ lại: 272

--- Features được chọn: ---
                                              feature  importance
109                                  EXT_SOURCES_MEAN    0.022812
111                                   EXT_SOURCES_MIN    0.017340
112                                   EXT_SOURCES_MAX    0.016145
28                                       EXT_SOURCE_2    0.014417
6                                          DAYS_BIRTH    0.008653
..                                                ...         ...
142     NAME_EDUCATION_TYPE_Secondarysecondaryspecial    0.000910
530  POS_NAME_CONTRACT_STATUS_RETURNEDTOTHESTORE_MEAN    0.000908
179                WEEKDAY_APPR_PROCESS_START_TUESDAY    0.000906
121                                 FLAG_OWN_REALTY_Y    0.000899
274                 BUREA

## 10. Xử lý Missing value, Cân bằng dữ liệu

In [23]:
# Thư viện cần thiết cho các bước tiếp theo
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

In [25]:
print("--- BƯỚC 10: Xử lý NaN, Chia Train/Test & Cân bằng Dữ liệu ---")

# Sửa đổi logic để giữ lại SK_ID_CURR trong tập features khi điền NaN

# 1. Định nghĩa lại tập train và test
train_df_filtered = df_final[df_final['TARGET'].notnull()].copy()
test_df_filtered = df_final[df_final['TARGET'].isnull()].copy()
print(f"Kích thước tập Train đã lọc (vẫn còn NaN): {train_df_filtered.shape}")


# 2. XỬ LÝ MISSING VALUE TRÊN TẬP TRAIN (Imputation) 
X_all_features = train_df_filtered.drop(columns=['TARGET']).copy()
y_train = train_df_filtered['TARGET']

X_features_only = X_all_features.drop(columns=['SK_ID_CURR'])

X_filled_features = X_features_only.fillna(X_features_only.mean())

for col in X_filled_features.select_dtypes(include=['object']).columns:
    X_filled_features[col] = X_filled_features[col].fillna(X_filled_features[col].mode()[0]) 

# Thêm lại SK_ID_CURR vào tập features đã fill
X_filled_features['SK_ID_CURR'] = X_all_features['SK_ID_CURR'].values 

# clean_df giờ đây sẽ bao gồm tất cả features, SK_ID_CURR, và TARGET
clean_df = pd.concat(
    [X_filled_features.reset_index(drop=True), y_train.reset_index(drop=True)], 
    axis=1
)


# 3. CHIA LẠI TRAIN/TEST 
X = clean_df.drop(columns=["TARGET", "SK_ID_CURR"])
y = clean_df["TARGET"]

# Chia dữ liệu train/test (80% - 20%)
X_train_full, X_test_full, y_train_full, y_test_full = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Giảm kích thước (Sampling, theo logic của AI_PROJECT.ipynb)
train_frac = 0.2 
test_frac = 0.1

X_train_small = X_train_full.sample(frac=train_frac, random_state=42)
y_train_small = y_train_full.loc[X_train_small.index]

X_test_small = X_test_full.sample(frac=test_frac, random_state=42)
y_test_small = y_test_full.loc[X_test_small.index]

print(f"\nKích thước tập train nhỏ: {X_train_small.shape}, Test nhỏ: {X_test_small.shape}")

# 4. CÂN BẰNG DỮ LIỆU (UnderSampling)
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train_small, y_train_small)

print("✅ Train nhỏ đã được cân bằng bằng UnderSampling. Kích thước mới:")
print(pd.Series(y_train_resampled).value_counts())

# 5. XỬ LÝ MISSING VALUE TRÊN TẬP TEST (Áp dụng Mean từ tập Train gốc)
# Tách X_test_final (tập test nhỏ) và xử lý NaN bằng Mean từ TẬP TRAIN gốc (X_train)
X_test_final = X_test_small.fillna(X_train.mean()) # Dùng mean của tập train gốc

# Gộp lại thành DataFrame Test cuối cùng (đã clean)
test_df_final = pd.concat(
    [X_test_final.reset_index(drop=True),
     y_test_small.reset_index(drop=True)],
    axis=1
)

# 6. XUẤT FILE CUỐI CÙNG
train_balanced_df = pd.concat(
    [pd.DataFrame(X_train_resampled, columns=X.columns),
     pd.Series(y_train_resampled, name='TARGET')],
    axis=1
)

# Sử dụng một đường dẫn chung (PATH đã định nghĩa ở đầu file)
output_path_train_balanced = PATH + "/train_balanced_under.csv"
output_path_test_small = PATH + "/test_modeling_small.csv" # Đổi tên cho rõ ràng hơn

train_balanced_df.to_csv(output_path_train_balanced, index=False)
test_df_final.to_csv(output_path_test_small, index=False)


print("\n--- HOÀN TẤT QUY TRÌNH FEATURE ENGINEERING & TIỀN MÔ HÌNH ---")
print(f"💾 Train balanced đã lưu: {train_balanced_df.shape}")
print(f"💾 Test modeling đã lưu: {test_df_final.shape}")

--- BƯỚC 10: Xử lý NaN, Chia Train/Test & Cân bằng Dữ liệu ---
Kích thước tập Train đã lọc (vẫn còn NaN): (307511, 545)

Kích thước tập train nhỏ: (49202, 543), Test nhỏ: (6150, 543)
✅ Train nhỏ đã được cân bằng bằng UnderSampling. Kích thước mới:
TARGET
0.0    3958
1.0    3958
Name: count, dtype: int64

--- HOÀN TẤT QUY TRÌNH FEATURE ENGINEERING & TIỀN MÔ HÌNH ---
💾 Train balanced đã lưu: (7916, 544)
💾 Test modeling đã lưu: (6150, 544)
